In [ ]:
#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
#Loading the dataset
df= pd.read_excel("ENRSheartfailuredata.xlsx")
df.head()

,Death,Age,EdemaScore,Parocysmal_Dyspnea_Or_Orthopnea,Dyspnea_on_exertion,Fatigue,Chest_Pain,Systolic_BP,Diastolic_BP,HeartRate,RespiratoryRate,LVEF,TmeToDiuretic_Log
0,0.0,65,2,0.0,1.0,0.0,0,169.0,120.0,98.0,24.0,22,2.212188
1,0.0,76,3,1.0,1.0,1.0,0,124.0,74.0,69.0,20.0,37,2.071882
2,1.0,92,1,0.0,1.0,1.0,0,117.0,64.0,87.0,18.0,70,NaN
3,1.0,77,3,0.0,1.0,0.0,0,98.0,60.0,68.0,18.0,44,2.033424
4,0.0,96,1,1.0,1.0,1.0,0,135.0,105.0,144.0,18.0,50,2.471292


In [ ]:
# Checking the missing values
df.isnull().sum()

Death                               1
Age                                 0
EdemaScore                          0
Parocysmal_Dyspnea_Or_Orthopnea    52
Dyspnea_on_exertion                50
Fatigue                            50
Chest_Pain                          0
Systolic_BP                        50
Diastolic_BP                       50
HeartRate                          50
RespiratoryRate                    50
LVEF                                0
TmeToDiuretic_Log                   3
dtype: int64

In [ ]:
df.fillna(0, inplace = True)


Classify the 25th,50,75th,100 percentile into groups as 1,2,3,4

In [ ]:
# df.loc[df['TP'].between(0.381000, 0.88800, inclusive = 'right'), 'quartile'] = 1

# df.loc[df['TP'].between(0.79925, 0.88800, inclusive = 'right'), 'quartile'] = 2
# df.loc[df['TP'].between(0.88800, 0.940250, inclusive = 'right'), 'quartile'] = 3
# df.loc[df['TP'].between(0.940250, 1, inclusive = 'right'), 'quartile'] = 4

In [ ]:
df

,Death,Age,EdemaScore,Parocysmal_Dyspnea_Or_Orthopnea,Dyspnea_on_exertion,Fatigue,Chest_Pain,Systolic_BP,Diastolic_BP,HeartRate,RespiratoryRate,LVEF,TmeToDiuretic_Log
0,0.0,65,2,0.0,1.0,0.0,0,169.0,120.0,98.0,24.0,22,2.212188
1,0.0,76,3,1.0,1.0,1.0,0,124.0,74.0,69.0,20.0,37,2.071882
2,1.0,92,1,0.0,1.0,1.0,0,117.0,64.0,87.0,18.0,70,0.000000
3,1.0,77,3,0.0,1.0,0.0,0,98.0,60.0,68.0,18.0,44,2.033424
4,0.0,96,1,1.0,1.0,1.0,0,135.0,105.0,144.0,18.0,50,2.471292
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,35,1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,29,2.392697
96,0.0,50,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,69,2.920123
97,0.0,56,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,61,0.000000
98,0.0,75,2,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,68,2.419956


In [ ]:
df.isnull().sum()
df.dropna(inplace=True)

Drop all null values

RANDOM FOREST


TAKE 10 % test data and 90% train data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Death', 1), df['Death'],test_size=0.1)


In [ ]:
X_train

,Age,EdemaScore,Parocysmal_Dyspnea_Or_Orthopnea,Dyspnea_on_exertion,Fatigue,Chest_Pain,Systolic_BP,Diastolic_BP,HeartRate,RespiratoryRate,LVEF,TmeToDiuretic_Log
98,75,2,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,68,2.419956
62,46,2,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,35,3.185542
59,85,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,40,2.356026
85,73,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,42,2.334454
90,74,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,61,3.641177
...,...,...,...,...,...,...,...,...,...,...,...,...
11,73,3,1.0,0.0,1.0,0,168.0,64.0,87.0,22.0,69,2.285557
25,71,2,1.0,0.0,0.0,1,128.0,70.0,76.0,0.0,65,2.960946
9,87,2,1.0,1.0,0.0,1,122.0,62.0,87.0,18.0,51,2.527630
7,49,1,0.0,0.0,1.0,0,127.0,77.0,97.0,16.0,20,2.324282


In [ ]:
y_train.shape

(90,)

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7

In [ ]:
pd.DataFrame({
    'features': df.drop('Death', 1).columns,
    'importance': clf.feature_importances_
})

,features,importance
0,Age,0.214854
1,EdemaScore,0.064842
2,Parocysmal_Dyspnea_Or_Orthopnea,0.023064
3,Dyspnea_on_exertion,0.012916
4,Fatigue,0.019712
5,Chest_Pain,0.032158
6,Systolic_BP,0.071501
7,Diastolic_BP,0.070959
8,HeartRate,0.062495
9,RespiratoryRate,0.066864


In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
# Number of trees in random forest
n_estimators = [5,10,40]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [3, 4]
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [10,20, 25]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 6, 8]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [3, 4],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [2, 4, 6, 8],
 'min_samples_split': [10, 20, 25],
 'n_estimators': [5, 10, 40]}


# RandomForestRegressor

In [ ]:

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(df.drop('Death', 1), df['Death'])

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [3, 4],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [2, 4, 6, 8],
                                        'min_samples_split': [10, 20, 25],
                                        'n_estimators': [5, 10, 40]},
                   random_state=42, verbose=2)

In [ ]:
rf_random.best_params_

{'n_estimators': 5,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 4,
 'bootstrap': True}

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, df.drop('Death', 1), df['Death'])

Model Performance
Average Error: 0.2479 degrees.
Accuracy = -inf%.


In [ ]:
rf_random.best_estimator_.feature_importances_

array([0.35795438, 0.01808583, 0.        , 0.        , 0.02754222,
       0.00399834, 0.09446489, 0.00055939, 0.02180954, 0.04038449,
       0.07862672, 0.3565742 ])

In [ ]:
rf_random.best_estimator_.feature_names_in_

array(['Age', 'EdemaScore', 'Parocysmal_Dyspnea_Or_Orthopnea',
       'Dyspnea_on_exertion', 'Fatigue', 'Chest_Pain', 'Systolic_BP',
       'Diastolic_BP', 'HeartRate', 'RespiratoryRate', 'LVEF',
       'TmeToDiuretic_Log'], dtype=object)

## Examine the Default Random Forest to Determine Parameters


In [ ]:
# Number of trees in random forest
n_estimators = [50,100]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [5,10, 15,20]
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,5,8,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 6, 8]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [5, 10, 15, 20],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [2, 4, 6, 8],
 'min_samples_split': [2, 5, 8, 10],
 'n_estimators': [50, 100]}


## Random Search Training of RandomForestClassifier
Now, we instantiate the random search and fit it like any Scikit-Learn model:

In [ ]:

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(df.drop('Death', 1), df['Death'])

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, 10, 15, 20],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 8, 10],
                                        'n_estimators': [50, 100]},
                   random_state=42, verbose=2)

# Evaluation Function


In [ ]:
rf_random.best_estimator_

RandomForestClassifier(max_depth=15, min_samples_leaf=8)

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, df.drop('Death', 1), df['Death'])

Model Performance
Average Error: 0.2100 degrees.
Accuracy = 0.00%.


In [ ]:
impr = pd.DataFrame(rf_random.best_estimator_.feature_importances_, index=rf_random.best_estimator_.feature_names_in_, columns=['importance']).sort_values('importance', ascending=False)
display(impr)
print(impr.index)

,importance
Age,0.212523
TmeToDiuretic_Log,0.190305
LVEF,0.115261
Diastolic_BP,0.090192
RespiratoryRate,0.084132
EdemaScore,0.067017
HeartRate,0.064256
Chest_Pain,0.057043
Systolic_BP,0.045869
Parocysmal_Dyspnea_Or_Orthopnea,0.037329


Index(['Age', 'TmeToDiuretic_Log', 'LVEF', 'Diastolic_BP', 'RespiratoryRate',
       'EdemaScore', 'HeartRate', 'Chest_Pain', 'Systolic_BP',
       'Parocysmal_Dyspnea_Or_Orthopnea', 'Fatigue', 'Dyspnea_on_exertion'],
      dtype='object')


## USING ONLY TOP 7 FEATURES

In [ ]:
rf_random.best_estimator_

RandomForestClassifier(max_depth=15, min_samples_leaf=8)